## Utils

### Imports

In [1]:
import requests
from environments.alfworld.common.environment import ALFWorldEnv

import os
from langchain_openai import ChatOpenAI
from planning_library.strategies import ADaPTStrategy
from planning_library.strategies.adapt.components import ADaPTPlanner, ADaPTExecutor
from planning_library.action_executors import LangchainActionExecutor, MetaTools
from planning_library.utils.gym_env_reset_tool import GymEnvResetTool
from textwrap import dedent

%load_ext autoreload
%autoreload 2

### Logging

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "test"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

## Environment

In [3]:
os.environ["ALFWORLD_DATA"] = "utils/data"

if not "base_config.yaml" in os.listdir("utils"):
    response = requests.get("https://raw.githubusercontent.com/alfworld/alfworld/master/configs/base_config.yaml")
    
    with open("utils/base_config.yaml", "wb") as f:
        f.write(response.content)

env = ALFWorldEnv(config_path="utils/base_config.yaml")
meta_tools = MetaTools(reset=GymEnvResetTool(env=env))
action_executor = LangchainActionExecutor(env.tools, meta_tools=meta_tools)

Initializing AlfredTWEnv...


100%|██████████| 8810/8810 [00:05<00:00, 1712.34it/s]

Overall we have 3553 games in split=train
Training with 3553 games


In [4]:
obs, info = env.reset(options={"next_episode": True})
print(obs)

-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.

Your task is to: put some pencil on shelf.


## Strategy

### Hyperparameters

In [5]:
# ADaPT hyperparameters
max_depth = 3  # maximum decomposition depth
executor_max_iterations = 2  # maximum number of iterations for the executor; when reached, the loop will exit
planner_max_iterations = 20  # maximum number of iterations for the planner; when reached, the loop will exit

# other hyperparameters
model_name = "gpt-3.5-turbo"
temperature = 0.8

### Executor

Default Executor is powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).

In [6]:
executor = ADaPTExecutor.create_simple_strategy(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                             parser_name="openai-tools",
                                                user_message=dedent("""
            You are ALFRED, an intelligent agent navigating in a household.

            {inputs}"""),      max_iterations=executor_max_iterations,
                                                return_intermediate_steps=True,
                                                return_finish_log=True,
                                                action_executor=action_executor)

### Planner

Planner can either be:

* `agent`: llm with tools, can be powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).
* `simple`: a single call to llm.

In [7]:
agent_planner = ADaPTPlanner.create_agent_planner(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                             parser_name="openai-tools", 
                                                user_message=dedent("""
            Here is the essence of the original task:

            {inputs}
            """),
            executor_parser_name="openai-tools")

In [8]:
simple_planner = ADaPTPlanner.create_simple_planner(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                                user_message=dedent("""
            Here is the essence of the original task:

            {inputs}
            """),
            executor_parser_name="openai-tools")

## Defining strategy

In [9]:
agent_planner_adapt = ADaPTStrategy(
    executor=executor,
    planner=agent_planner,
    max_depth=max_depth,
    return_intermediate_steps=True,
    return_finish_log=True,
)

In [10]:
agent_planner_adapt.invoke(
    {"inputs": {"inputs": obs}}
)



> Entering new ADaPTStrategy chain...


> Entering new SimpleStrategy chain...
I encountered an issue while trying to put the pencil on the shelf. Let me try again to complete the task.

> Finished chain.


> Entering new SimpleStrategy chain...
I have added the first subtask to the plan: "Go to the shelf 1 and place a pencil on it."  
The plan is set to aggregate the results with an "and" logic, meaning all subtasks must be successfully completed.  

> Finished chain.


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...
The step-by-step plan to successfully solve the task "Go to the shelf 1 and place a pencil on it" with "and" logic aggregation is as follows:

1. Go to shelf 1.
2. Place a pencil on shelf 1.

This plan ensures that both subtasks are executed sequentially, and the task will only be considered successfully solved if both subtasks are completed successfully in the specified order.

> Finished chain.


> Entering new Simpl

{'inputs': {'inputs': '-= Welcome to TextWorld, ALFRED! =-\n\nYou are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.\n\nYour task is to: put some pencil on shelf.'},
 'intermediate_steps': [[]],
 'finish_log': ["Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Maximum decomposition depth reached."]}

In [11]:
simple_planner_adapt = ADaPTStrategy(
    executor=executor,
    planner=simple_planner,
    max_depth=max_depth,
    return_intermediate_steps=True,
    return_finish_log=True,
)

In [12]:
simple_planner_adapt.invoke(
    {"inputs": {"inputs": obs}}
)



> Entering new ADaPTStrategy chain...


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...
task completed

> Finished chain.


> Entering new SimpleStrategy chain...
It seems like I couldn't put the pencil on shelf 1. Let me try again.

> Finished chain.


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...


> Finished chain.
Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Maximum decomposition depth reached.

> Finished chain.


{'inputs': {'inputs': '-= Welcome to TextWorld, ALFRED! =-\n\nYou are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.\n\nYour task is to: put some pencil on shelf.'},
 'intermediate_steps': [[(OpenAIToolAgentAction(tool='goto', tool_input={'receptable_type': 'sidetable', 'receptable_id': 1}, log="\nInvoking: `goto` with `{'receptable_type': 'sidetable', 'receptable_id': 1}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_zVKtRn26Dhvg6cESJx9Xfk54', 'function': {'arguments': '{"receptable_type":"sidetable","receptable_id":1}', 'name': 'goto'}, 'type': 'function'}]})], tool_call_id='call_zVKtRn26Dhvg6cESJx9Xfk54'),
    AgentStep(action=OpenAIToolAgentAction(tool='goto', tool_input={'receptable_type': 'sidetable', 'receptable_id': 1}, log="\nInvoking: `goto` with `{'receptable_typ